In [3]:
import numpy as np

In [194]:
inputm = (input('enter patient rankings separated by commas in the following format: patient 1 rankings;patient 2 rankings;...;patient n rankings\n\n').split(sep=';'))
capacity = int(input('\nenter doctor capacity\n'))
input_matrix = np.fromstring(inputm[0],int,sep=',')
for i in range(1,len(inputm)):
    input_matrix = np.vstack((input_matrix, np.fromstring(inputm[i],int,sep=',')))

enter patient rankings separated by commas in the following format: patient 1 rankings;patient 2 rankings;...;patient n rankings

1,2,3,4;4,3,2,1;2,4,1,3;3,1,2,4;4,2,3,1;2,3,1,4;1,4,3,2

enter doctor capacity
2


In [14]:
# repeat columns as needed for doctor capacity and add dummy variables to make square matrix, 
# with colnames = each spot for doctor, rownames = patients
p = int(input_matrix.shape[0])
number_of_doctor = int(input_matrix.shape[1])

matrix = np.repeat(input_matrix,capacity,axis=1)
dum=0
while(matrix.shape[0] < matrix.shape[1]):
    dum+=1
    matrix=np.vstack([matrix,[0]*matrix.shape[1]])
matrix=matrix.transpose()
matrix_for_check = matrix.copy()

# subtract the minimum value in each row from its row and the minimum value in each column from its column 
# in order to make the smallest entry in each row and each column = 0
dim=matrix.shape[0]
cur_mat=matrix
for i in range(dim):
    cur_mat[i]=cur_mat[i]-np.min(cur_mat[i])
for i in range(dim):
    cur_mat[:,i]=cur_mat[:,i]-np.min(cur_mat[:,i])

def count_zero(cur_mat,matrix_minimum):
# function which tallies how many times the overall minimum of the matrix appears in each row and each column
    num_zero_row=[]
    for i in range(cur_mat.shape[0]):
        row_zero = np.where(cur_mat[i].astype(int)==matrix_minimum)[0]
        num_zero_row.append(len(row_zero))

    num_zero_col=[]
    for i in range(cur_mat.shape[1]):
        col_zero = np.where(cur_mat[:,i].astype(int)==matrix_minimum)[0]
        num_zero_col.append(len(col_zero))

    return num_zero_col,num_zero_row

# Use the number of dimensions of the matrix of lines to cover all the minimum value in the matrix
def optimality_test(tmp_mat,line):
# def optimality_test is used to test how many lines needed to cover all the minimum value
    row_delete=[]
    row_remain=list(np.arange(0,dim))
    col_remain=list(np.arange(0,dim))
    col_delete=[]
    matrix_minimum=np.min(tmp_mat)
    num_zero_col,num_zero_row = count_zero(tmp_mat,matrix_minimum)
    result = False  
    while(1):
        if result == False:
            if np.max(num_zero_row) >= np.max(num_zero_col):
                tmp_row_delete=np.where(np.array(num_zero_row) == np.max(num_zero_row))[0][0]
                tmp_mat[tmp_row_delete]+=1
                row_delete.append(tmp_row_delete)
                row_remain.remove(tmp_row_delete)
                num_zero_col,num_zero_row = count_zero(tmp_mat,matrix_minimum)
                result=np.all((np.array(num_zero_row) == 0))+np.all((np.array(num_zero_col) == 0))
            elif np.max(num_zero_col) > np.max(num_zero_row):
                tmp_col_delete=np.where(np.array(num_zero_col) == np.max(num_zero_col))[0][0]
                tmp_mat[:,tmp_col_delete]+=1
                col_delete.append(tmp_col_delete)
                col_remain.remove(tmp_col_delete)
                num_zero_col,num_zero_row = count_zero(tmp_mat,matrix_minimum)
                result=np.all((np.array(num_zero_row) == 0))+np.all((np.array(num_zero_col) == 0))
            line+=1
        else:
            break
    return tmp_mat,row_delete,col_delete,row_remain,col_remain,line

tmp_mat=cur_mat.copy()
line=1
tmp_mat,row_delete,col_delete,row_remain,col_remain,line = optimality_test(tmp_mat,line)


# If there are less lines needed, get the minimum value of the uncovered values, and 
# subtract it from the uncovered values and add it to the intercepts of the lines

while(line < dim):
    tmp_mat,row_delete,col_delete,row_remain,col_remain,line = optimality_test(tmp_mat,line)
    min_value=np.min(tmp_mat)
    for i in row_delete:
        for l in col_delete:
            cur_mat[i,l]+=min_value
    for x in row_remain:
        for y in col_remain:
            cur_mat[x,y]-=min_value
    tmp_mat=cur_mat.copy()

##assignment follow the following two tools depending on the minimum vlaue of the whole matrix:
## 1: if there is any rows or columns that exists only 1 minimum value, then take that point
## 2: if there are multiple minimum value in rows and columns, pick the first point of rows or columns having the least number of minimum values
cur_mat=cur_mat[:,:-dum]
i=1
final_row_index=[]
final_col_index=[]

count=0
for count in range(p):
    matrix_minimum=np.min(cur_mat)
    num_zero_col,num_zero_row=count_zero(cur_mat,matrix_minimum)
    if 1 in num_zero_row:
        tmp_row=np.where(np.array(num_zero_row) == 1)[0][0]
        tmp_col = np.where(cur_mat[tmp_row]==min(cur_mat[tmp_row]))[0][0]
        cur_mat[tmp_row] += 999
        cur_mat[:,tmp_col] += 999
        final_row_index.append(tmp_row)
        final_col_index.append(tmp_col)
    elif 1 in num_zero_col:
        tmp_col=np.where(np.array(num_zero_col) == 1)[0][0]
        tmp_row=np.where(cur_mat[:,tmp_col]==min(cur_mat[:,tmp_col]))[0][0]
        cur_mat[tmp_row] += 999
        cur_mat[:,tmp_col] += 999
        final_row_index.append(tmp_row)
        final_col_index.append(tmp_col) 
    else:
        min_col=np.min(list(filter(lambda x: x != 0, num_zero_col)))
        min_row=np.min(list(filter(lambda x: x != 0, num_zero_row)))
        if min_row <= min_col:
            tmp_row=np.where(np.array(num_zero_row) == min_row)[0][0]
            tmp_col=np.where(cur_mat[tmp_row]==min(cur_mat[tmp_row]))[0][0]
            cur_mat[tmp_row] += 999
            cur_mat[:,tmp_col] += 999
            final_row_index.append(tmp_row)
            final_col_index.append(tmp_col)
        else:
            tmp_col=np.where(np.array(num_zero_col) == min_col)[0][0]
            tmp_row=np.where(cur_mat[:,tmp_col]==min(cur_mat[:,tmp_col]))[0][0]
            cur_mat[tmp_row] += 999
            cur_mat[:,tmp_col] += 999
            final_row_index.append(tmp_row)
            final_col_index.append(tmp_col)
            
    i+=1
    count+=1

        
            
# output optimal solution
start=0
for i in range(number_of_doctor):
    end= start+capacity
    l = list(range(start, end))
    start=end
    patient_index = np.array(np.where(np.isin(final_row_index,l))[0])
    patient_num=np.array(final_col_index)[np.array(patient_index.astype(int))]
    print("doctor "+ str(i+1)+" has the following patients: "+ str(patient_num+1))

doctor 1 has the following patients: [1 7]
doctor 2 has the following patients: [4]
doctor 3 has the following patients: [3 6]
doctor 4 has the following patients: [2 5]
